# Deploy COA Objects in Transcend:   DBInfo (Demo Project)

In [ ]:
%addconnect name=Transcend, user=sh186014, host=tdprdcop3.td.teradata.com, props="LOGMECH=LDAP"

In [2]:
%connect name=Transcend

Password ············


Success: 'Transcend' connection established


In [ ]:
call adlste_coa.sp_meta_objects_refresh('DBCInfo') 

## Stage Object (adlste_coa_stg)

In [7]:
drop table adlste_coa_stg.stg_dat_dbcinfo;

Success: 22 rows affected

In [8]:
CREATE MULTISET GLOBAL TEMPORARY TABLE adlste_coa_stg.stg_dat_DBCinfo
( Account_Name   VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, Site_ID        VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, InfoKey        VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, InfoData       VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
) NO PRIMARY INDEX 
ON COMMIT PRESERVE ROWS;

Success: 0 rows affected

In [11]:
select count(*) from adlste_coa_stg.stg_dat_dbcinfo;

Count(*)
0


## Target Table and View (adlste_coa)

In [ ]:
DROP TABLE adlste_coa.coat_dat_DBCinfo;

In [ ]:
CREATE MULTISET TABLE adlste_coa.coat_dat_DBCinfo
( Account_Name   VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, Site_ID        VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, LogDate        DATE Default Current_Date
, InfoKey        VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, InfoData       VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
) Primary Index(Site_ID, LogDate);

In [ ]:
REPLACE VIEW adlste_coa.coa_dat_DBCinfo
as locking row for access
Select * from adlste_coa.coat_dat_DBCinfo;

## Stored Proc to manage the merge

In [3]:
REPLACE PROCEDURE adlste_coa.sp_dat_dbcinfo ( spversion VARCHAR(128) ) 
SQL SECURITY CREATOR
BEGIN 

    /* start setup audit process */
    
    DECLARE runid  INTEGER;
    DECLARE siteid VARCHAR(128);
    DECLARE tablename VARCHAR(128);
    DECLARE callingsp VARCHAR(128); 
    DECLARE startdate DATE;
    DECLARE enddate DATE;
    
    SET runid = 0;
    SET tablename = 'adlste_coa.coat_dat_dbcinfo';
    SET callingsp = 'adlste_coa.sp_dat_dbcinfo';
    SET siteid = 'Empty Table';
    SET startdate = DATE;
    SET enddate = DATE;

    SELECT 
     coalesce(Site_ID,'empty table')       
    ,current_date as StartDate
    ,current_date as EndDate
    INTO siteid, startdate, enddate
    FROM adlste_coa_stg.stg_dat_dbcinfo 
    Group by  1,2,3
    ;
    
    /* end setup audit process */
    
    
    
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'START', '') ;

    DELETE FROM adlste_coa.coat_dat_dbcinfo WHERE (Site_ID, LogDate) IN
    (Select Site_ID, Current_Date From adlste_coa_stg.stg_dat_dbcinfo );
    
    Insert into adlste_coa.coat_dat_dbcinfo 
    Select Account_Name, Site_ID, Current_Date, InfoKey, InfoData 
    from adlste_coa_stg.stg_dat_dbcinfo
    group  by 1,2,3,4,5; 
        
    delete from adlste_coa_stg.stg_dat_dbcinfo;
    
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'END', '') ;

END;

Success: 0 rows affected

In [ ]:
-- Generate Test Data
insert into adlste_coa_stg.stg_dat_DBCinfo
select 'test account', 'testsiteid123', infokey, infodata from dbc.dbcinfo;

In [12]:
call adlste_coa.sp_dat_dbcinfo('1.1')

Success: 0 rows affected

In [ ]:
select * from adlste_coa.coa_audit_runlog order by 1

In [ ]:
select * from adlste_coa.coa_audit_summary order by 1

## Report Success

In [24]:
select * 
from adlste_coa.coa_audit_summary
where LogDate = DATE
  and CallingSP = 'adlste_coa.sp_dat_dbcinfo'
order by 1 desc

Run_ID,Site_ID,LogDate,TableName,CallingSP,SQLVersion,Run_Type,Steps,Errors,Warnings,Start_TS,End_TS
77,Test_SiteID,2020-05-22,adlste_coa.coat_dat_dbcinfo,adlste_coa.sp_dat_dbcinfo,1.1,normal run,2,0,0,2020-05-22 19:25:21,2020-05-22 19:25:21
73,Transcend_IFX,2020-05-22,adlste_coa.coat_dat_dbcinfo,adlste_coa.sp_dat_dbcinfo,1.15,normal run,2,0,0,2020-05-22 19:18:44,2020-05-22 19:18:45
75,Empty Table,2020-05-22,adlste_coa.coat_dat_dbcinfo,adlste_coa.sp_dat_dbcinfo,1.1,normal run,2,0,0,2020-05-22 19:19:30,2020-05-22 19:19:30


## Test the Process

In [21]:
insert into adlste_coa_stg.stg_dat_dbcinfo 
Select 'Test_Account',  'Test_SiteID', InfoKey, InfoData from dbc.dbcinfo;

Select * from adlste_coa_stg.stg_dat_dbcinfo 

Success: 3 rows affected

Account_Name,Site_ID,InfoKey,InfoData
Test_Account,Test_SiteID,RELEASE,16.20.32.11
Test_Account,Test_SiteID,VERSION,16.20.32.11
Test_Account,Test_SiteID,LANGUAGE SUPPORT MODE,Standard


In [22]:
call adlste_coa.sp_dat_dbcinfo('1.1');

Success: 0 rows affected

In [23]:
Select * from adlste_coa.coa_dat_dbcinfo 

Account_Name,Site_ID,LogDate,InfoKey,InfoData
Demo Customer,transcend,2020-05-13,VERSION,16.20.32.11
test account,testsiteid123,2020-05-07,LANGUAGE SUPPORT MODE,Standard
Test_Account,Test_SiteID,2020-05-22,LANGUAGE SUPPORT MODE,Standard
Demo Customer,trancend_target_01,2020-05-20,RELEASE,16.20.32.11
Demo Customer,trancend_target_01,2020-05-13,RELEASE,16.20.32.11
Demo Customer,trancend_target_01,2020-05-19,VERSION,16.20.32.11
test account,testsiteid123,2020-05-08,LANGUAGE SUPPORT MODE,Standard
Testing,Transcend_IFX,2020-05-22,LANGUAGE SUPPORT MODE,Standard
Demo Customer,tdtranscend,2020-05-17,LANGUAGE SUPPORT MODE,Standard
Demo Customer,trancend_target_01,2020-05-11,LANGUAGE SUPPORT MODE,Standard


In [6]:
show table adlste_coa_stg.stg_dat_dbcinfo

Request Text
"CREATE MULTISET TABLE adlste_coa_stg.stg_dat_dbcinfo ,FALLBACK , NO BEFORE JOURNAL, NO AFTER JOURNAL, CHECKSUM = DEFAULT, DEFAULT MERGE"
